In [1]:
import os
import pandas as pd

from network_wrangler import ProjectCard

from update_registry import update_registry
from methods_io import read_project_cards
from methods_add_cards import add_cards_to_registry
from methods_add_cards import _make_available

## Input/Output

In [2]:
CARD_DIR = os.path.join(".", "projects")
REGISTRY_FILE = "registry.csv"
CONFIG_FILE = "registry_config.yml"

## Update Registry

In [3]:
update_registry(
        CONFIG_FILE, REGISTRY_FILE, "delete_me.csv", CARD_DIR, False,
    )

## read in card as dict and inspect

In [ ]:
dirpath = CARD_DIR
filename = "year_2015_san_mateo_bridge_add_plaza_link.yml"
card_file = os.path.join(dirpath, filename)
card = ProjectCard.read(card_file, validate=False)

In [ ]:
card_dict = card.__dict__
card_dict.keys()

In [ ]:
changes_dict = card_dict["changes"]
changes_dict[1]

In [ ]:
"changes" in card_dict.keys()

In [ ]:
changes_list = card_dict["changes"]
for change_dict in changes_list:
    print("here")

In [ ]:
card_dict.get("donkey")

In [ ]:
for change_dict in changes_list:
    print(change_dict)

In [ ]:
for index, change_dict in enumerate(changes_list):
    print(index)

In [ ]:
"changes" in card_dict

In [ ]:
"category" in card_dict

In [ ]:
"category" in change_dict

In [ ]:
for i in range(7,12 + 1):
    print(i)

In [ ]:
ids_in_use = {n: False for n in range(1, 10 + 1)}
ids_in_use

In [ ]:
for i in range(1,10+1):
    ids_in_use[i] = True

ids_in_use